In [1]:
include("load_pod.jl");
include("load_s1slc_ann.jl");
include("calc_sat_trajectory.jl");
include("llh2sali.jl")

llh2sali (generic function with 1 method)

In [2]:
llh = [
    [(56 + 33/60 + 42.5556/(60*60)),(8 + 18/60 + 26.874/(60*60)), 42.753],
    [(56 + 33/60 + 44.4528/(60*60)) ,(8 + 18/60 + 24.865/(60*60)),43.282],
    [(56 + 33/60 + 42.8652/(60*60)) ,(8 + 18/60 + 22.104/(60*60)),44.475],
    [(56.003092) ,8.129702 ,43.5]
];
llh_cat = vcat(llh'...);

In [3]:
lat_max = 5.666818777336966e+01
lon_max = 8.692126035601964e+00;

In [4]:
lat_min = 5.514243624845066e+01
lon_min = 6.863612098279713e+00;

In [5]:
lat = collect(range(lat_min, length=1000, stop=lat_max));
lon = collect(range(lon_min, length=1000, stop=lon_max));

In [6]:
test_points = [[lat_i,lon_i,rand()*20+20]  for lat_i in lat, lon_i in lon]
test_points=reshape(test_points,:);
test_points_cat = vcat(test_points'...);

In [7]:
f = open("ann_path.txt")
path = readlines(f)
sar_parameters = load_s1slc_ann(path[1]);

In [11]:
sar_parameters

Dict{String,Any} with 20 entries:
  "image_number"          => "006"
  "mode"                  => "IW"
  "mission_data_id"       => "33712"
  "burst_meta"            => Dict{String,Any}("first_line_mosaic"=>[1, 1343, 26…
  "samples_per_burst"     => 25330
  "geolocation"           => Dict{String,Any}("line"=>[0, 0, 0, 0, 0, 0, 0, 0, …
  "polarisation"          => "VV"
  "absolute_orbit_number" => "4715"
  "slant_range_time"      => 0.00602248
  "t_start"               => 50.7031
  "azimuth_frequency"     => 486.486
  "t_stop"                => 78.6587
  "pass"                  => "Descending"
  "mission_id"            => "S1B"
  "product_type"          => "SLC"
  "burst_count"           => 10
  "range_pixel_spacing"   => 2.32956
  "lines_per_burst"       => 1524
  "t_0"                   => 2017-03-15T05:39:00
  "swath"                 => 3

In [8]:
sar_parameters["geolocation"]

Dict{String,Any} with 4 entries:
  "line"      => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  15240, 15240, 15240, 15240, …
  "latitude"  => [56.6682, 56.6749, 56.6816, 56.6882, 56.6947, 56.7012, 56.7075…
  "longitude" => [8.69213, 8.62024, 8.54869, 8.47746, 8.40655, 8.33594, 8.26564…
  "sample"    => [0, 1267, 2534, 3801, 5068, 6335, 7602, 8869, 10136, 11403  … …

## Test

In [9]:
f = open("POD_path.txt")
path = readlines(f)
osv,t_sv = load_pod(path[1],sar_parameters["t_0"]);

In [10]:
@time llh2sali(llh_cat, osv, t_sv, sar_parameters)

  3.369375 seconds (10.91 M allocations: 547.149 MiB, 9.67% gc time)


4×2 Array{Float64,2}:
 1126.43  6086.27
 1122.66  6098.7 
 1126.71  6109.43
 5685.96  6255.93

In [12]:
@time llh2sali(test_points_cat, osv, t_sv, sar_parameters);

  9.308887 seconds (182.00 M allocations: 16.124 GiB, 13.91% gc time)
